<a href="https://colab.research.google.com/github/ValentinoRyo10/Machine-Learning/blob/main/CareerMatch_MLmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>